In [8]:
langjson = {
  "nodes": [
    {
      "id": "__START__",
      "description": "Entry point of the graph.",
    },
    {
      "id": "planner",
      "schema_info": "PlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)",
      "input_schema": "PlanExecute",
      "output_schema": "Plan",
      "description": "Generates a step-by-step plan based on user input.",
      "function_name": "plan_step"
    },
    {
      "id": "agent",
      "schema_info": "PlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)",
      "input_schema": "PlanExecute",
      "output_schema": "PlanExecute",
      "description": "Executes the first step of the plan and updates past steps.",
      "function_name": "execute_step"
    },
    {
      "id": "replan",
      "schema_info": "PlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)",
      "input_schema": "PlanExecute",
      "output_schema": "Union[Response, Plan]",
      "description": "Updates the plan based on current state and past steps.",
      "function_name": "replan_step"
    },
    {
      "id": "__END__",
      "description": "End point of the graph.",
    }
  ],
  "edges": [
    {
      "source": "__START__",
      "target": "planner",
      "routing_conditions": "Start the planning process.",
      "conditional": False
    },
    {
      "source": "planner",
      "target": "agent",
      "routing_conditions": "After planning, execute the first step.",
      "conditional": False
    },
    {
      "source": "agent",
      "target": "replan",
      "routing_conditions": "After executing a step, check if replanning is needed.",
      "conditional": False
    },
    {
      "source": "replan",
      "target": "agent",
      "routing_conditions": "If no response is generated, continue to agent for further execution.",
      "conditional": False
    },
    {
      "source": "replan",
      "target": "__END__",
      "routing_conditions": "If a response is generated, end the process.",
      "conditional": True
    }
  ]
}

In [17]:
import json

def dict_to_tree_positions(langjson):
    reactflow = {"nodes": [], "edges": []}
    nodes = langjson.get("nodes", [])
    edges = langjson.get("edges", [])

    # Create a mapping of edges to track parent-child relationships
    children_map = {}
    for edge in edges:
        parent = edge["source"]
        child = edge["target"]
        if parent not in children_map:
            children_map[parent] = []
        children_map[parent].append(child)

    # Helper function to assign positions recursively
    def assign_positions(node_id, depth, x_offset, x_step, visited):
        # Prevent infinite recursion by checking if the node is already visited
        if node_id in visited:
            return
        visited.add(node_id)

        if node_id not in node_positions:
            node_positions[node_id] = {"x": x_offset, "y": depth * y_spacing}
            # Add node to ReactFlow nodes
            node = next((n for n in nodes if n["id"] == node_id), None)
            if node:
                reactflow["nodes"].append({
                    "id": node_id,
                    "data": {
                        "description": node.get("description", ""),
                        "function_name": node.get("function_name", ""),
                        "label": node_id.replace("_", " ").title()
                    },
                    "position": {"x": x_offset, "y": depth * y_spacing}
                })

        # Recurse to children
        if node_id in children_map:
            num_children = len(children_map[node_id])
            child_x_offset = x_offset - (x_step * (num_children - 1)) / 2
            for index, child_id in enumerate(children_map[node_id]):
                assign_positions(
                    child_id, depth + 1, child_x_offset + (index * x_step), x_step // 2, visited
                )

    # Root node position configuration
    y_spacing = 150  # Vertical distance between levels
    x_step = 400  # Horizontal distance between child nodes
    node_positions = {}

    # Assume __START__ is the root node
    visited_nodes = set()  # Track visited nodes to avoid infinite loops
    assign_positions("__START__", 0, 0, x_step, visited_nodes)

    # Add regular edges to ReactFlow
    for edge in edges:
        reactflow["edges"].append({
            "id": f'{edge["source"]}_to_{edge["target"]}',
            "source": edge["source"],
            "target": edge["target"],
            "animated": edge.get("conditional", False)  # Set animated if conditional is True
        })

    # Convert to JSON with double quotes
    return json.dumps(reactflow, indent=2)


In [16]:
print(dict_to_tree_positions(langjson))

{
  "nodes": [
    {
      "id": "__START__",
      "data": {
        "description": "Entry point of the graph.",
        "function_name": "",
        "label": "  Start  "
      },
      "position": {
        "x": 0,
        "y": 0
      }
    },
    {
      "id": "planner",
      "data": {
        "description": "Generates a step-by-step plan based on user input.",
        "function_name": "plan_step",
        "label": "Planner"
      },
      "position": {
        "x": 0.0,
        "y": 150
      }
    },
    {
      "id": "agent",
      "data": {
        "description": "Executes the first step of the plan and updates past steps.",
        "function_name": "execute_step",
        "label": "Agent"
      },
      "position": {
        "x": 0.0,
        "y": 300
      }
    },
    {
      "id": "replan",
      "data": {
        "description": "Updates the plan based on current state and past steps.",
        "function_name": "replan_step",
        "label": "Replan"
      },
      "posit